<a href="https://colab.research.google.com/github/yarengokhn/Bist100TrandingAlgorithm/blob/main/ProjectTable100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance
!pip install mplfinance
!pip install numpy
!pip install keras
!pip install pandas_ta
!pip install catboost
!pip install optuna
!pip install xgboost

import yfinance as yf
import pandas as pd
from datetime import date

import optuna
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import pandas_ta as ta
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

start_date = '2022-01-01'
end_date = date.today().strftime("%Y-%m-%d")

# Bist 100 ticker list  array
bistList = ['AEFES.IS', 'AGHOL.IS', 'AHGAZ.IS', 'AKBNK.IS', 'AKCNS.IS', 'AKFGY.IS', 'AKFYE.IS', 'AKSA.IS', 'AKSEN.IS', 'ALARK.IS', 'ALBRK.IS', 'ALFAS.IS', 'ARCLK.IS', 'ASELS.IS', 'ASTOR.IS', 'BERA.IS', 'BIENY.IS', 'BIMAS.IS', 'BRSAN.IS', 'BRYAT.IS', 'BUCIM.IS', 'CANTE.IS', 'CCOLA.IS', 'CIMSA.IS', 'CWENE.IS', 'DOAS.IS', 'DOHOL.IS', 'ECILC.IS', 'ECZYT.IS', 'EGEEN.IS', 'EKGYO.IS', 'ENJSA.IS', 'ENKAI.IS', 'EREGL.IS', 'EUPWR.IS', 'EUREN.IS', 'FROTO.IS', 'GARAN.IS', 'GENIL.IS', 'GESAN.IS', 'GLYHO.IS', 'GUBRF.IS', 'GWIND.IS', 'HALKB.IS', 'HEKTS.IS', 'IMASM.IS', 'IPEKE.IS', 'ISCTR.IS', 'ISDMR.IS', 'ISGYO.IS', 'ISMEN.IS', 'IZMDC.IS', 'KARSN.IS', 'KAYSE.IS', 'KCAER.IS', 'KCHOL.IS', 'KMPUR.IS', 'KONTR.IS', 'KONYA.IS', 'KORDS.IS', 'KOZAA.IS', 'KOZAL.IS', 'KRDMD.IS', 'KZBGY.IS', 'MAVI.IS', 'MGROS.IS', 'MIATK.IS', 'ODAS.IS', 'OTKAR.IS', 'OYAKC.IS', 'PENTA.IS', 'PETKM.IS', 'PGSUS.IS', 'PSGYO.IS', 'QUAGR.IS', 'SAHOL.IS', 'SASA.IS', 'SISE.IS', 'SKBNK.IS', 'SMRTG.IS', 'SNGYO.IS', 'SOKM.IS', 'TAVHL.IS', 'TCELL.IS', 'THYAO.IS', 'TKFEN.IS', 'TOASO.IS', 'TSKB.IS', 'TTKOM.IS', 'TTRAK.IS', 'TUKAS.IS', 'TUPRS.IS', 'ULKER.IS', 'VAKBN.IS', 'VESBE.IS', 'VESTL.IS', 'YEOTK.IS', 'YKBNK.IS', 'YYLGD.IS', 'ZOREN.IS']
ticker_accuracies = []
ticker_precisions = []
ticker_recalls = []
ticker_f1s = []
tickers = []

models = ['XGBClassifier','CatBoost', 'RandomForest', 'LightGBM','LSTM']
model_accuracies = []
model_precisions = []
model_recalls = []
model_f1s = []
model_names = []

i=1
n=len(bistList)
for ticker in bistList:

  print(f"----------------------\n Ticker number: {i}----------------------------------\n")
  i=i+1
  #Storing data into data frame
  try:
    df = yf.download(ticker , start_date, end_date)

    #Calculating MFI(Money Flow Index)
    df.ta.mfi(append=True)

    #Calculating RSI
    df.ta.rsi(append=True)

    #Calculating EMA and SMA Technical Indicators
    emas_and_smas_lengths = [5,8,9,13,50,200]
    for length in emas_and_smas_lengths:
      df.ta.ema(length=length, append=True)
      df.ta.sma(length=length, append=True)

    #Calculating Bollinger Bands
    df.ta.bbands(append=True)

    #Calculating RSI
    df.ta.rsi(append=True)

    #Calculating MFI(Money Flow Index)
    df.ta.mfi(append=True)

    #Calculating MACD
    df.ta.macd(append=True)


    #Calculating Ichimoku
    df.ta.ichimoku(append=True)

    #Calculating ADX
    df.ta.adx(append=True)

    # Measures the relationship between price and average price over a specific time.
    df.ta.cci(append=True)

    # #Calculating ATR
    df.ta.atr(append=True)

    standard_deviation_time = 30 #Standard Deviation for a month

    df.ta.stdev(length=standard_deviation_time, append=True)


    #macd
    df_features = pd.DataFrame()
    df_features["MACD_12_26_9"] = df ["MACD_12_26_9"].to_frame().copy()
    df_features["MACDh_12_26_9"] = df ["MACDh_12_26_9"].to_frame().copy()
    df_features["MACDs_12_26_9"] = df ["MACDs_12_26_9"].to_frame().copy()
    df_features['MACD_Signal'] = (df['MACD_12_26_9'] > df['MACDs_12_26_9']).astype(int)

    #macd with 34,144,9
    df.ta.macd(fast=34, slow=144, signal=9, append=True)
    df_features['MACD_34_144_9'] = df['MACD_34_144_9'].to_frame().copy()
    df_features['MACDh_34_144_9'] = df['MACDh_34_144_9'].to_frame().copy()
    df_features['MACDs_34_144_9'] = df['MACDs_34_144_9'].to_frame().copy()
    #bbands
    df_features['BBL_5_2.0'] = df['BBL_5_2.0'].to_frame().copy()
    df_features['BBM_5_2.0'] = df['BBM_5_2.0'].to_frame().copy()
    df_features['BBU_5_2.0'] = df['BBL_5_2.0'].to_frame().copy()
    df_features['BBB_5_2.0'] = df['BBB_5_2.0'].to_frame().copy()
    df_features


    #ADX
    df_features["ADX_14"]=df ["ADX_14"].to_frame().copy()

    df_features["RSI_30"] = np.where(df["RSI_14"] < 30,1,0)
    df_features["RSI_70"] = np.where(df["RSI_14"] > 70,1,0)
    df_features["Close"] = df["Close"].to_frame().copy()
    df_features["EMA_5"] = df["EMA_5"].to_frame().copy()
    df_features["SMA_9_50"] = np.where(df["SMA_9"] > df["SMA_13"],1,0)
    #df_features["SMA_13_50"] = np.where(df["SMA_13"] > df["SMA_50"],1,0)
    #df_features["SMA_50_200"] = np.where(df["SMA_50"] > df["SMA_200"],1,0)
    #df_features["EMA_5_SMA_8"] = np.where(df["EMA_5"] > df["SMA_8"],1,0)
    df_features["Bollinger_Upper_Band_%"]=df['BBU_5_2.0'] - df['Close'] / df['Close']
    df_features["Signal_MFI_50"] = np.where(df["MFI_14"] < 50 ,1,0)
    df_features['target'] = np.where(df['Open'].shift(-1) - df['Close']>0 ,1,0)


    df_features = df_features.dropna()
    X = df_features.drop(['target'],axis=1).values
    y = df_features.target.values
    train_size = int(len(X)*0.71)
    X_train, X_test = X[0:train_size], X[train_size:len(X)]
    y_train, y_test = y[0:train_size], y[train_size:len(X)]

    scalar=MinMaxScaler()
    X_train=scalar.fit_transform(X_train)
    X_test=scalar.fit_transform(X_test)

#------------------XGBClassifier(without optuna)-------------------------------
    model = XGBClassifier(learning_rate=0.05, n_estimators=500, subsample=1.0, alpha=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)


    accuracy_xgb = accuracy_score(y_test, y_pred)
    precision_xgb = precision_score(y_test, y_pred, average='binary')
    recall_xgb = recall_score(y_test, y_pred, average='binary')
    f1_xgb = f1_score(y_test, y_pred, average='weighted')


#---------------------------CatBoost (without optuna)------------------------------------------
    model = CatBoostClassifier(iterations=500, depth=6, learning_rate=0.1, loss_function='Logloss', random_state=42,verbose=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)


    accuracy_catboost = accuracy_score(y_test, y_pred)
    precision_catboost = precision_score(y_test, y_pred, average='binary')
    recall_catboost = recall_score(y_test, y_pred, average='binary')
    f1_catboost = f1_score(y_test, y_pred, average='weighted')

#-------------------------------RandomForest(without optuna)----------------------------------------------

    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)


    accuracy_rf = accuracy_score(y_test, y_pred)
    precision_rf = precision_score(y_test, y_pred, average='binary')
    recall_rf = recall_score(y_test, y_pred, average='binary')
    f1_rf = f1_score(y_test, y_pred, average='weighted')


  #-------------------------LightGBM(with optuna)-------------------------------
    def objectiveLGBM(trial):

      params = {
          'n_estimators': trial.suggest_int('n_estimators', 10, 100),
          'max_depth': trial.suggest_int('max_depth', 2, 10),
          'learning_rate': trial.suggest_float('learning_rate', 0.02, 0.3),
          'min_child_samples': trial.suggest_int('min_child_samples', 2, 10),
          'random_state': 42,
          'num_leaves': trial.suggest_int('num_leaves',5 , 15),
          "verbose":-1
      }

      model = LGBMClassifier(**params)

      model.fit(X_train, y_train)
      y_pred = model.predict(X_test)
      accuracy = accuracy_score(y_test, y_pred)

      return accuracy

    study = optuna.create_study(direction='maximize')
    study.optimize(objectiveLGBM, n_trials=20)

    best_params = study.best_params

    best_model = LGBMClassifier(**best_params)

    best_model.fit(X_train, y_train)
    y_pred_best = best_model.predict(X_test)

    accuracy_lgbm = accuracy_score(y_test, y_pred_best)
    precision_lgbm = precision_score(y_test, y_pred_best, average='binary')
    recall_lgbm = recall_score(y_test, y_pred_best, average='binary')
    f1_lgbm = f1_score(y_test, y_pred_best, average='weighted')
   # -----------------------------LSTM(without optuna)-------------------------
    model = Sequential()
    model.add(LSTM(100, activation="tanh", return_sequences=True, input_shape=(1, X_train.shape[1])))
    model.add(LSTM(49, activation="tanh"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer="rmsprop", loss='binary_crossentropy', metrics=['accuracy'])

    X_train = np.resize(X_train, (X_train.shape[0],1,X_train.shape[1]))
    X_test = np.resize(X_test, (X_test.shape[0],1,X_test.shape[1]))
    model.fit(X_train, y_train, epochs=8, batch_size=len(X_train), validation_data=(X_test, y_test), verbose=0)
    threshold = 0.525
    y_pred = model.predict(X_test).T[0]
    y_pred = (y_pred > threshold).astype("int32")


    accuracy_lstm = accuracy_score(y_test, y_pred)
    precision_lstm = precision_score(y_test, y_pred, average='binary')
    recall_lstm = recall_score(y_test, y_pred, average='binary')
    f1_lstm = f1_score(y_test, y_pred, average='weighted')


    #--------------------------------------------------------------------------------

    highest_f1_model = models[np.argmax([f1_xgb,f1_catboost, f1_rf, f1_lgbm,f1_lstm])]


    if highest_f1_model == 'CatBoost':
            accuracy = accuracy_catboost
            precision = precision_catboost
            recall = recall_catboost
            f1 = f1_catboost
    elif highest_f1_model == 'RandomForest':
        accuracy = accuracy_rf
        precision = precision_rf
        recall = recall_rf
        f1 = f1_rf
    elif highest_f1_model == 'XGBClassifier':
        accuracy = accuracy_xgb
        precision = precision_xgb
        recall = recall_xgb
        f1 = f1_xgb
    elif highest_f1_model == 'LSTM':
        accuracy = accuracy_lstm
        precision = precision_lstm
        recall = recall_lstm
        f1 = f1_lstm
    else:
        accuracy = accuracy_lgbm
        precision = precision_lgbm
        recall = recall_lgbm
        f1 = f1_lgbm

    ticker_accuracies.append(accuracy)
    ticker_precisions.append(precision)
    ticker_recalls.append(recall)
    ticker_f1s.append(f1)
    tickers.append(ticker)
    model_names.append(highest_f1_model)

  except Exception as e:
    continue

table = pd.DataFrame({
    "Stock": tickers,
    "Accuracy": ticker_accuracies,
    "Precision": ticker_precisions,
    "Recall": ticker_recalls,
    "F1": ticker_f1s,
    "Model": model_names
})

table


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 494.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=b807aea97cf35b7ced0ac4e02d25bec262de7edd05bcd29865bdef222e7abcd1
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.8 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


----------------------
 Ticker number: 1----------------------------------



[*********************100%%**********************]  1 of 1 completed
[I 2024-01-28 08:20:50,413] A new study created in memory with name: no-name-32e69126-547c-4b21-869f-6d3d7332fba2
[I 2024-01-28 08:20:50,485] Trial 0 finished with value: 0.6296296296296297 and parameters: {'n_estimators': 46, 'max_depth': 5, 'learning_rate': 0.1631363508884511, 'min_child_samples': 2, 'num_leaves': 5}. Best is trial 0 with value: 0.6296296296296297.
[I 2024-01-28 08:20:50,512] Trial 1 finished with value: 0.6296296296296297 and parameters: {'n_estimators': 82, 'max_depth': 4, 'learning_rate': 0.10036239288965378, 'min_child_samples': 3, 'num_leaves': 11}. Best is trial 0 with value: 0.6296296296296297.
[I 2024-01-28 08:20:50,533] Trial 2 finished with value: 0.6851851851851852 and parameters: {'n_estimators': 94, 'max_depth': 2, 'learning_rate': 0.284334500562749, 'min_child_samples': 8, 'num_leaves': 9}. Best is trial 2 with value: 0.6851851851851852.
[I 2024-01-28 08:20:50,546] Trial 3 finished wit

[LightGBM] [Info] Number of positive: 182, number of negative: 79
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000171 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1225
[LightGBM] [Info] Number of data points in the train set: 261, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.697318 -> initscore=0.834559
[LightGBM] [Info] Start training from score 0.834559
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

...

[*********************100%%**********************]  1 of 1 completed
[I 2024-01-28 08:37:58,135] A new study created in memory with name: no-name-dd92e5ed-1cfe-4226-89d2-cca021280b86
[I 2024-01-28 08:37:58,169] Trial 0 finished with value: 0.4875 and parameters: {'n_estimators': 53, 'max_depth': 6, 'learning_rate': 0.058926495982114996, 'min_child_samples': 3, 'num_leaves': 15}. Best is trial 0 with value: 0.4875.
[I 2024-01-28 08:37:58,211] Trial 1 finished with value: 0.45 and parameters: {'n_estimators': 86, 'max_depth': 9, 'learning_rate': 0.19085665808132074, 'min_child_samples': 2, 'num_leaves': 13}. Best is trial 0 with value: 0.4875.
[I 2024-01-28 08:37:58,233] Trial 2 finished with value: 0.425 and parameters: {'n_estimators': 25, 'max_depth': 10, 'learning_rate': 0.2690844548072987, 'min_child_samples': 5, 'num_leaves': 12}. Best is trial 0 with value: 0.4875.
[I 2024-01-28 08:37:58,254] Trial 3 finished with value: 0.425 and parameters: {'n_estimators': 45, 'max_depth': 10, 

[LightGBM] [Info] Number of positive: 115, number of negative: 79
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000096 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 930
[LightGBM] [Info] Number of data points in the train set: 194, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.592784 -> initscore=0.375484
[LightGBM] [Info] Start training from score 0.375484
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

[*********************100%%**********************]  1 of 1 completed
[I 2024-01-28 08:38:09,596] A new study created in memory with name: no-name-5ba23bc4-cb97-455c-9fa9-513f9c3310ea
[I 2024-01-28 08:38:09,633] Trial 0 finished with value: 0.5648148148148148 and parameters: {'n_estimators': 100, 'max_depth': 4, 'learning_rate': 0.21325770610069095, 'min_child_samples': 10, 'num_leaves': 13}. Best is trial 0 with value: 0.5648148148148148.
[I 2024-01-28 08:38:09,660] Trial 1 finished with value: 0.5740740740740741 and parameters: {'n_estimators': 46, 'max_depth': 4, 'learning_rate': 0.2485087589644429, 'min_child_samples': 10, 'num_leaves': 15}. Best is trial 1 with value: 0.5740740740740741.
[I 2024-01-28 08:38:09,683] Trial 2 finished with value: 0.5833333333333334 and parameters: {'n_estimators': 25, 'max_depth': 9, 'learning_rate': 0.2929474140348629, 'min_child_samples': 6, 'num_leaves': 9}. Best is trial 2 with value: 0.5833333333333334.
[I 2024-01-28 08:38:09,703] Trial 3 finishe

[LightGBM] [Info] Number of positive: 164, number of negative: 97
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000144 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1219
[LightGBM] [Info] Number of data points in the train set: 261, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.628352 -> initscore=0.525155
[LightGBM] [Info] Start training from score 0.525155
4/4 [==============================] - 1s 5ms/step


,Stock,Accuracy,Precision,Recall,F1,Model
0,AEFES.IS,0.694444,0.705263,0.930556,0.643774,RandomForest
1,AGHOL.IS,0.592593,0.580645,0.915254,0.529940,LightGBM
2,AHGAZ.IS,0.500000,0.517241,0.789474,0.447917,XGBClassifier
3,AKBNK.IS,0.666667,0.669903,0.971831,0.570334,LightGBM
4,AKCNS.IS,0.564815,0.589744,0.754098,0.543387,RandomForest
...,...,...,...,...,...,...
87,VESBE.IS,0.601852,0.617978,0.859375,0.555383,LightGBM
88,VESTL.IS,0.574074,0.555556,0.818182,0.545575,LightGBM
89,YKBNK.IS,0.583333,0.630952,0.791045,0.552301,LSTM
90,YYLGD.IS,0.500000,0.638298,0.566038,0.510000,XGBClassifier


In [ ]:
best_tickers_table = table.sort_values(by='F1', ascending=False)

top_10_tickers = best_tickers_table.head(10)


top_10_table = pd.DataFrame({
    "Stock": top_10_tickers["Stock"],
    "Accuracy": top_10_tickers["Accuracy"],
    "Precision": top_10_tickers["Precision"],
    "Recall": top_10_tickers["Recall"],
    "F1": top_10_tickers["F1"],
    "Model": top_10_tickers["Model"]
})

top_10_table

,Stock,Accuracy,Precision,Recall,F1,Model
13,BIENY.IS,0.833333,1.000000,0.800000,0.851852,LightGBM
34,GENIL.IS,0.764706,0.909091,0.769231,0.778431,LightGBM
22,DOAS.IS,0.777778,0.777778,0.974684,0.731783,LightGBM
49,KCHOL.IS,0.750000,0.780220,0.910256,0.725041,LightGBM
61,ODAS.IS,0.722222,0.750000,0.907895,0.689153,LightGBM
67,QUAGR.IS,0.703704,0.728395,0.855072,0.688620,LightGBM
36,GLYHO.IS,0.740741,0.757576,0.949367,0.687080,LSTM
12,BERA.IS,0.712963,0.741935,0.907895,0.675489,RandomForest
65,PETKM.IS,0.712963,0.731183,0.918919,0.673641,LightGBM
25,ECZYT.IS,0.750000,0.769231,0.963855,0.673520,RandomForest
